#            Stock Market Quarterly Investment Recommender

A Data Science Project by Eric J Campbell for The Data Incubator 2019 NYC Fall Cohort

# Motivation

In today's modern and fast-paced world, both investors and financial companies alike are looking for great investment opportunities in the stock market. But it is by no means easy to predict what company will be the next Google, Apple, or Tesla.

Instead of investing large amounts of capital in a small company and waiting 30 years to see if the investment was worth the money, many companies and individuals have set their eyes on the short-term, where money could be made just from buying or selling a stock at the right point in time.

Large banks collect as much data as they can to try and predict if a stock will rise or fall, yet this information can be expensive, in addition to being the product of teams of scientists working for months or even years to construct a predictive model.

What hope does the average investor have?

The purpose of this capstone project is to take advantage of a seasonal occurrence among publicly-traded companies. As required by law, any company on the stock exchange must disclose a quarterly report which details how the company performed for those three months of the year.

One particular KPI is the Earnings Per Share, or simply EPS. The EPS is a measure of how well the company earned a profit, and is therefore a good metric to determine how well a company performed in the previous quarter. Comparing the EPS to the predicted EPS yields the Surprise. A positive Surprise usually results in investors flocking to buy stock in the company, resulting in large gains in stock price. The reverse is usually the case for a negative Surprise, with investors quickly selling the stock so their money could yield larger returns elsewhere.

A prime example of these semi-predictable swings in stock price after a quarterly earnings report release can be seen in the plot below, which shows the stock price for Tesla over the last two years, along with the points where a quarterly report was issued. Both positive Surprise (green dots) and negative Surprise (red dots) are seen. Most of the time, the EPS report generates a predictable outcome!

In [ ]:
plot_tesla_chart()

At this point, you may be wondering how an investor could benefit financially if the contents of the quarterly report are private until everyone else and their uncle knows about it.

The advantage introduced here is through the use of a Facebook dataset which tracks the check-ins, likes, and talking about counts for various companies over time. Given this data for companies with a physical presence, the check-ins and other features can be used to see how well a company is doing at attracting people - and therefore customers, to its location. Check-ins have been successfully used in the past to predict the drop in Chipotle stock due to an E-Coli problem in its food.

With this information, predictions can be made to determine if a company is expected to have a positive or negative Surprise, resulting in a simple output from the recommender engine: buy this stock, or sell this stock. If you did not own the stock prior to the quarterly release, you might enjoy a nice spike in stock price, and then sell for a short-term profit, or keep for long-term potential. If you did own the stock, you may have a avoided a painful drop in your investment portfolio by selling beforehand. Furthermore, you could always buy the stock after the predicted fall at a good price.

With that being said, it is important to note that check-ins and the other features from the Facebook dataset are not a crystal ball. Unforseen occurrences are always a possibility, such as corporate scandals, lawsuits, heavy infrastructure spending (and therefore little to no profit), etc...). As with any good financial advice, it is important to diversify to avoid such risks.

In [20]:
import pandas as pd
df = pd.read_csv('/home/jovyan/temp_datalab_records_social_facebook.csv', parse_dates=True, low_memory=False)

In [21]:
def clean_dataframe(dataframe):
    """Deletes unwanted columns from dataframe,
    Converts date column to datetime.date object
    Strips and converts username column to uppercase"""
    
    dataframe.time = pd.to_datetime(dataframe.time, format='%Y-%m-%d')
    dataframe.time = [x.date() for x in dataframe.time]
    
    del dataframe['date_added']
    del dataframe['date_updated']
    del dataframe['entity_id']
    del dataframe['cusip']
    del dataframe['isin']
    del dataframe['has_added_app']
    del dataframe['facebook_id']
    del dataframe['were_here_count']
    
    dataframe.username = dff.username.str.upper()
    dataframe.username = dff.username.str.strip()
    
    return dataframe

In [22]:
dff = clean_dataframe(df)

In [23]:
dff.head()

,dataset_id,time,username,checkins,likes,talking_about_count
0,53088,2015-01-01,SODASTREAM,0,761533,587
1,52642,2015-01-01,ANSYSINC,148,43442,1305
2,53656,2015-01-01,MYAQUAAMERICA,0,5591,297
3,53033,2015-01-01,QUALCOMM,173,1232521,2685
4,52783,2015-01-01,EAGLEPHARMACEUTICALS,0,18,0


In [27]:
def get_unique_companies(dataframe):
    """Generates list of unique companies in dataframe"""
    clist = list(dataframe.username.unique())
    while np.nan in clist:
        clist.remove(np.nan)
    return clist

In [28]:
facebook_companies = get_unique_companies(dff)

In [29]:
facebook_companies

['SODASTREAM',
 'ANSYSINC',
 'MYAQUAAMERICA',
 'QUALCOMM',
 'EAGLEPHARMACEUTICALS',
 'BIODELIVERYSI',
 'UMBBANK',
 'SILICOM',
 'GSK',
 'OFFICEDEPOT',
 'CTWTR',
 'TDAMERITRADE',
 'IMAGESENSINGSYS',
 'OFFICIALCSX',
 'BLACKROCK',
 'GOPRESTO',
 'CHICOPEESAVINGS',
 'CSGINTERNATIONAL',
 'HEXCELCORP',
 'SERVICENOW',
 'NEPTUNEKRILLOIL',
 'TRIPOINTEHOMES',
 'AEGON',
 '8X8INC',
 'ACIWORLDWIDE',
 'NORTHRIMBANK',
 'ADDUSHOMECARE',
 'PRIMOWATERCORP',
 'SEAGATE',
 'BRANDYWINEREALTYTRUST',
 'LIGHTINTHEBOX',
 'MYHORIZONBANK',
 'OCCSOLUTIONS',
 'CORNINGINCORPORATED',
 'CAPITALSENIORLIVING',
 'EXELIS',
 'CARPENTERTECH',
 'BUSEYBANK',
 'AIRPRODUCTSCAREERS',
 'PHIINC',
 'STAMPS.COM',
 'MARCHEX',
 'ADT',
 'MURPHYUSA',
 'BANCOSANTANDER',
 'TIVO',
 'RFMDMEDIA',
 'ICFINTERNATIONAL',
 'APPLIEDGENETICTECHNOLOGIESCORPORATION',
 'AMDOCS',
 'VPGINC',
 'LENNOX',
 'ABBVIECAREERS',
 'TOTAL',
 'SABRECORPORATION',
 'FARMERSBANKGROUP',
 'POWERINTEGRATIONS',
 'REXNORDCORPORATE',
 'HFRECIPES',
 'EFI.DIGITAL.PRINT.TECHNOLO

In [32]:
def isolate_physical_companies(dataframe, company_list):
    """Removes companies from company_list where 
    the max or min checkins is equal to 0, or where 
    the checkins have not changes, which is indicative of
    a non-physical company"""
    physical_company = []
    
    for company in company_list:
        
        df = dataframe[dataframe.username == company]
        if (np.max(df.checkins) == 0) or (np.min(df.checkins) == 0):
            pass
        elif np.max(df.checkins) == np.min(df.checkins):
            pass
        else:
            physical_company.append(company)
    return physical_company

In [33]:
physical_companies = isolate_physical_companies(dff, facebook_companies)

In [34]:
physical_companies

['ANSYSINC',
 'QUALCOMM',
 'UMBBANK',
 'GSK',
 'CTWTR',
 'IMAGESENSINGSYS',
 'CHICOPEESAVINGS',
 'CSGINTERNATIONAL',
 'HEXCELCORP',
 'SERVICENOW',
 'TRIPOINTEHOMES',
 'AEGON',
 'NORTHRIMBANK',
 'ADDUSHOMECARE',
 'PRIMOWATERCORP',
 'BRANDYWINEREALTYTRUST',
 'LIGHTINTHEBOX',
 'MYHORIZONBANK',
 'OCCSOLUTIONS',
 'CAPITALSENIORLIVING',
 'EXELIS',
 'BUSEYBANK',
 'AIRPRODUCTSCAREERS',
 'PHIINC',
 'STAMPS.COM',
 'MARCHEX',
 'RFMDMEDIA',
 'ICFINTERNATIONAL',
 'APPLIEDGENETICTECHNOLOGIESCORPORATION',
 'LENNOX',
 'ABBVIECAREERS',
 'SABRECORPORATION',
 'FARMERSBANKGROUP',
 'POWERINTEGRATIONS',
 'REXNORDCORPORATE',
 'HFRECIPES',
 'EFI.DIGITAL.PRINT.TECHNOLOGY',
 'CHILDRENSPLACE',
 'RESOURCESGLOBALPROFESSIONALS',
 'OMAPLAZA',
 'ALIMERASCIENCES',
 'ASHLANDINC',
 'GREENBANKNA',
 'ROCKLANDTRUST',
 'IDENTIV',
 'PENNYMACUSA',
 'VALEROENERGY',
 'WALMART',
 'KAPSTONEPAPER',
 'ENGILITY',
 'EXPLORE.IHS',
 'HARTEHANKS',
 'PNCBANK',
 'WILLISGROUP',
 'STARBUCKSINDIA',
 'UNITEDCOMMUNITYBANKIN',
 'HIBBETTSPORTS',

In [59]:
import pickle
with open('physical_companies.pickle', 'wb') as handle:
    pickle.dump(physical_companies, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('physical_companies.pickle', 'rb') as handle:
#     physical_companies = pickle.load(handle)

In [515]:
def load_NYSE_NASDAQ_table():
    NSQEdf = pd.read_csv('NYSE_ticker_name.txt', sep='\t')
    NASDAQdf = pd.read_csv('NASDAQ_ticker_name.txt', sep='\t')

    NSQEdf.dropna(inplace=True)
    NASDAQdf.dropna(inplace=True)
    
    return pd.concat([NSQEdf, NASDAQdf])

In [516]:
stock_exchange_df = load_NYSE_NASDAQ_table()
stock_exchange_df.head()

,Symbol,Description
0,A,Agilent Technologies
1,AA,Alcoa Corp
2,AAC,Aac Holdings Inc
3,AAN,Aaron's Inc
4,AAP,Advance Auto Parts Inc


In [518]:
stock_exchange_companies = stock_exchange_df.Description.values
stock_exchange_tickers = stock_exchange_df.Symbol.values

In [519]:
stock_exchange_ticker_company = dict(zip(stock_exchange_tickers, stock_exchange_companies))
stock_exchange_company_ticker = dict(zip(stock_exchange_companies, stock_exchange_tickers))

In [521]:
stock_exchange_company_ticker

{'Agilent Technologies': 'A',
 'Alcoa Corp': 'AA',
 'Aac Holdings Inc': 'AAC',
 "Aaron's Inc": 'AAN',
 'Advance Auto Parts Inc': 'AAP',
 'American Assets Trust': 'AAT',
 'Alliancebernstein Holding LP': 'AB',
 'Abb Ltd': 'ABB',
 'Abbvie Inc': 'ABBV',
 'Amerisourcebergen Corp': 'ABC',
 'Ambev S.A.': 'ABEV',
 'Asbury Automotive Group Inc': 'ABG',
 'ABM Industries Incorporated': 'ABM',
 'Arbor Realty Trust': 'ABR',
 'Arbor Rlty TR Inc [Abr/Pa]': 'ABR-A',
 'Arbor Rlty TR Inc [Abr/Pb]': 'ABR-B',
 'Arbor Rlty TR Inc [Abr/Pc]': 'ABR-C',
 'Abbott Laboratories': 'ABT',
 'Associated Capital Group Inc': 'AC',
 'Arcosa Inc': 'ACA',
 'Aurora Cannabis Inc': 'ACB',
 'American Campus Communities Inc': 'ACC',
 'Acco Brands Corp': 'ACCO',
 'Aluminum Corp of China Ltd': 'ACH',
 'Aecom Technology Corp': 'ACM',
 'Accenture Plc': 'ACN',
 'Avenue Income Credit Strategies': 'ACP',
 'Ares Commercial Real Estate Cor': 'ACRE',
 'Allianzgi Diversified Income &': 'ACV',
 'Agree Realty Corp': 'ADC',
 'Archer Daniels

In [522]:
'WALMART' in stock_exchange_company_ticker.keys()

False

In [610]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process 

def best_name_match(fname, stock_names):
    """Given a list of possible company names matching the company name
    finds the best match """
    
    result = process.extractOne(fname, stock_names)

    return [result[0], result[1]]

In [605]:
best_name_match('GENERAL MOTORS', ['General Specific', 'Generally Great', 'General Motors'])

['General Motors', 100]

In [608]:
def get_stock_info(name):
    """Uses the Intrinio financial API to search for companies sounding similar to 
    the given company. Will return actual company name, ticker, and % certainty"""
    import requests
    import json
    import time
    
    names = []
    ticker = []
    
    def make_page_request():
        url = 'https://api-v2.intrinio.com/companies/search?query={}'.format(name)
        page = requests.get(url, params={'api_key':'Ojg5ZDFmOGNmMzJiOWZjM2RjZGNhNDRiM2JiNWJkM2M0'})
        return page
    
    status_code = 0
    while status_code != 200:
        page = make_page_request()
        status_code = page.status_code
        if status_code != 200:
            print('Failed with status code {}, retrying'.format(status_code))
            time.sleep(5)
        
    best_match = json.loads(page.text)

    for comp in best_match['companies']:
        names.append(comp['name'])
        ticker.append(comp['ticker'])

    if names:
        result = best_name_match(name, names)
    else:
        return {'name':np.nan, 'ticker':np.nan, 'certainty':np.nan}

    #return {'name':names[result[0]], 'ticker':ticker[result[0]], 'certainty':result[1]}
    return {'name':names[0], 'ticker':ticker[0], 'certainty':result[1]}

In [609]:
get_stock_info('WALMART')

{'name': 'Walmart Inc', 'ticker': 'WMT', 'certainty': 90}

In [611]:
def parse_all_companies(company_list):
    """For each comany in company_list, get actual company name
    and stock ticker"""
    
    import time
    company_master_list = {}
    
    for company in company_list:
        
        print(company)
        time.sleep(0.51)
        result = get_stock_info(company)
        
        if result['name'] != np.nan:
            company_master_list[company] = result
            
    return company_master_list

In [612]:
company_ticker_dict = parse_all_companies(physical_companies)

ANSYSINC
QUALCOMM
UMBBANK
GSK
CTWTR
IMAGESENSINGSYS
CHICOPEESAVINGS
CSGINTERNATIONAL
HEXCELCORP
SERVICENOW
TRIPOINTEHOMES
AEGON
NORTHRIMBANK
ADDUSHOMECARE
PRIMOWATERCORP
BRANDYWINEREALTYTRUST
LIGHTINTHEBOX
MYHORIZONBANK
OCCSOLUTIONS
CAPITALSENIORLIVING
EXELIS
BUSEYBANK
AIRPRODUCTSCAREERS
PHIINC
STAMPS.COM
MARCHEX
RFMDMEDIA
ICFINTERNATIONAL
APPLIEDGENETICTECHNOLOGIESCORPORATION
LENNOX
ABBVIECAREERS
SABRECORPORATION
FARMERSBANKGROUP
POWERINTEGRATIONS
REXNORDCORPORATE
HFRECIPES
EFI.DIGITAL.PRINT.TECHNOLOGY
CHILDRENSPLACE
RESOURCESGLOBALPROFESSIONALS
OMAPLAZA
ALIMERASCIENCES
ASHLANDINC
GREENBANKNA
ROCKLANDTRUST
IDENTIV
PENNYMACUSA
VALEROENERGY
WALMART
KAPSTONEPAPER
ENGILITY
EXPLORE.IHS
HARTEHANKS
PNCBANK
WILLISGROUP
STARBUCKSINDIA
UNITEDCOMMUNITYBANKIN
HIBBETTSPORTS
EQUINIX
WAGEWORKS
SEQUANS
HEARTLANDHPY
USGCORP
TERADATA
NEWSCORP
DELTA
ANGIESLIST
MACKCALIRC
TOPIMAGESYSTEMS
STREAMLINEHEALTHSOLUTIONS
PREMIERHEALTHCAREALLIANCE
SECONDSIGHTARGUS
RANGERESOURCESCORPORATION
DANAHOLDINGCORP
OGEPOWE

RAYGLASSNZ
COGGINDELANDLINCOLN
COURTESYPALMHARBORHONDA
NALLEYLEXUSGALLERIA
MOORMANSSHOWTEC
COGGINHONDAORLANDO
ATAGLANCE
NORTHPOINTTOYOTASCION
COGGINTOYOTAAVENUES
NORTHPOINTNISSAN
NALLEYINFINITIDECATUR
CYPRESSCAYPONTOONS
BEACONSALES
COGGINCHEVYJACKSONVILLE
COURTESYHYUNDAITAMPA
NORTHLITTLEROCKNISSAN
COGGINORANGEPARKBUICKGMC
MERCEDESBENZFORTPIERCE
BEDBATHANDBEYOND
NALLEYACURA
NALLEYHONDA
COURTESYKIABRANDON
NALLEYINFINITIMARIETTA
INFOSPACE
COGGINFORD
BMWOFLITTLEROCK
COURTESYCHRYSLERJEEPDODGETAMPA
MONOPRICECOM
ADMLLOYDMINSTER
NALLEYKIA
NORTHPOINTFORDLINCOLN
COGGINDELANDHONDA
MERCEDESBENZTAMPA
YOURALLDATATEAM
COGGINDELANDFORD
NORTHCOASTROOFINGSYSTEMS
KENSINGTONUS
COGGINNISSANAVENUES
INFINITITAMPA
NALLEYAUDINORTHATLANTA
SEARAYBOATSANDYACHTS
NISSANOFTAMPA
NORTHPOINTVOLVO
COGGINHONDAOFSTAUGUSTINE
TILIBRA
COGGINNISSANATLANTIC
LOCHINVARLLC
AUTOZONE
COGGINACURAFORTPIERCE
COGGINBMWTREASURECOAST
COGGINHONDAFORTPIERCE
NALLEYBMW
NALLEYLEXUSROSWELL
NORTHPOINTVW
SENSORYEFFECTS
ROOFCENTER
OXFORDCASINO
CA

ATTENTIONGLOBAL
MARINEMAXDALLAS
KOLOTV
MARINEMAXSANDIEGO
KWITTKEN
CBS6WTVR
FOX5SANDIEGO
ENTERPRISEBROCKTON
WHOHD
SUPREMEPROTEIN
CLAYCOUNTYADVOCATEPRESS
RECORDSTAR
BOONVILLEDAILYNEWS
BARNSTABLEPATRIOT
THEHELENAWORLD
BASTROPENTERPRISE
DEDHAMTRANSCRIPT
HNNOW
GLENROSEREPORTER
CANTONREP
THE.LEAVENWORTH.TIMES
PHL17
STEUBEN.COURIER
WELLINGTONDAILYNEWS
BROWNWOODBULLETIN
LEESVILLEDAILYLEADER
METROWESTDAILYNEWS
THECHRONICLEEXPRESS
EXAMINER.NET
TRICARE
THEOAKRIDGER
NEWSEAGLE
UTICAOD
SOUTHCOASTTODAY
TAUNTONTOGO
BUSHNELL
GURDONTIMES
SHAWNEENEWSSTAR
MTSHASTANEWS
SAULTEVENINGNEWS
JOURNALSTANDARD
MICHAELFOODS
CHILLICOTHECONSTITUTIONTRIBUNE
KIRKSVILLEDAILYEXPRESS
THEPROGRESSINDEX
WHITEHALLJOURNALNEWSPAPER
ALICEECHONEWS
BETTEROATS
PLAINRIDGEPARKCASINO
WGNRADIO
SEACOASTONLINE
TIMESREPORTER
SCSUNTIMES
REGISTERMAIL
ROCKFORDREGISTERSTAR
THESUNTIMES
THEWAYNEINDEPENDENT
LAKENEWSONLINE
THEDAILYLEADER
HILLSDALEDAILYNEWS
WAYNESVILLEDAILYGUIDE
RANDOLPHCOUNTYHERALDTRIBUNE
THEDAILYTELEGRAM
BILLCUNNINGHAMSHOW
PATRIO

USLCI
TEXASMOTORSPEEDWAY
LVMOTORSPEEDWAY
NHMOTORSPEEDWAY
STARTEKGLOBAL
WORKSPRINGCHICAGO
STEELCASEHEALTH
TYPTAP
SHINHANLIFE
TELEPHONICSCORPHQ
FIRSTMAJESTICSILVERBULLION
SJWATERCO
SKTWORLD
SKPLANET
HILLINTL
COLONYAMERICANFINANCE
FRANKSINTERNATIONAL
FTICONSULTINGINC
FLAGSTARBANK
JINKOSOLAR
RIGZONENEWS
BACHOCOMX
IONGEOPHYSICAL
SANTANDERMEX
GREENSTARENERGYUK
SPICERGELENKWELLENBAU
ICICIBANK
IDAHOPOWER
CRGIBSONGIFTS
EXZEO.INDIA
INVITAE
SIBANYERESOURCES
KITEREALTYGROUP
BERWICKOFFRAY
SANTAMARIATIMES
MATERIONCORP
MATTOONJGTC
SIOUXCITYJOURNAL
BEAVERDAMDAILYCITIZEN
HERALDANDREVIEW
JOURNALTIMES
BISTRIB
RCJOURNAL
WISCONSINSTATEJOURNAL
HANFORDSENTINEL
GTNEWS
MARCUSTHEATRES
THEFREMONTTRIBUNE
THETANDD
LACROSSETRIBUNE
MARCUSHOTELS
MITEL.NETWORKS
BILLINGSGAZETTE
ARIZONADAILYSUN
DCHUSA
MCGLOBEGAZETTE
DHNEWS
CUMBERLINK
WINONADAILYNEWS
PANTAGRAPH
LOMPOCRECORD
RAVALLIREPUBLIC
SOUTHERNILLINOISAN
ARIZONADAILYSTAR
NAPAVALLEYREGISTER
HELENAIRONLINE
MUSCATINEJOURNAL
THETIMESNEWS
STLPD
LINCOLNJOURNALSTAR
ELKODAIL

NEWTONPRESSMENTOR
MCPHERSONSENTINEL
PRATTTRIBUNEKS
CARMITIMES
HOPESTARAR
HAMBURGREPORTER
GENESEOREPUBLIC
KEWANEESTARCOURIER
PEKINDAILYTIMES
CHILLICOTHETIMESBULLETIN
MORNINGSUNKS
ROLLADAILYNEWS
ENTRAVISIONCOMMUNICATIONS
SIBANYESTILLWATER
GTTGUYANA
WICKEDLOCAL
MYOSRENSTECHNOLOGY
SEVERNBANK
MALIBUBOATS
NICEINCONTACT
FRESENIUSKIDNEYCARE
WAFDBANK
NORTHAMERICANADVANTAGEINSURANCE
SMARTSTOPSS
BUYDNOW
KENSINGTONTECH
HARDCAST65
BELOMEDIAGROUP
ALSIC9
MYHELLOREWARDS
VALLEYFAIRMN
APTIV
PURSUITGLACIERPARK
PURSUITALASKA
OASMIAPHARMACEUTICAL
PURSUITBANFFJASPER
AUTOPARTSWAREHOUSE
Failed with status code 502, retrying
BNBBANK
SAPCONCUR
GILEADSCIENCES
CLWSCTX
OTELCOINC
LATRICOLOR1059
DESTINATIONXL
LASUAVECITA1043
AVXCORP
IHSMARKIT
GOLDFIELDSLTD
ULTIMAIIID
COGGINFORDJAX
AMISPHILADELPHIA
POWEREDBYINAP
GAMESTOPITALIA
WESTLAKECHEMICAL
CAESARSENTERTAINMENTCORP
INTERCONTINENTALEXCHANGE
AMVBBDO
BBDOMPLS
PKPBBDO
BBDOSF
ALICEBBDO
PAGESBBDO
INTERONE.DE
CONTRAPUNTOBBDOESPANA
BBDOWARSZAWA
BBDOBELGIUM
SANCHOBBDO
PROX

In [613]:
company_ticker_dict

{'ANSYSINC': {'name': 'ANSYS Inc', 'ticker': 'ANSS', 'certainty': 94},
 'QUALCOMM': {'name': 'Qualcomm Inc', 'ticker': 'QCOM', 'certainty': 90},
 'UMBBANK': {'name': 'M&T Bank Corp', 'ticker': 'MTB', 'certainty': 77},
 'GSK': {'name': 'GlaxoSmithKline PLC', 'ticker': 'GSK', 'certainty': 60},
 'CTWTR': {'name': 'Twitter Inc', 'ticker': 'TWTR', 'certainty': 54},
 'IMAGESENSINGSYS': {'name': 'Image Sensing Systems Inc',
  'ticker': 'ISNS',
  'certainty': 78},
 'CHICOPEESAVINGS': {'name': 'Net Savings Link Inc',
  'ticker': 'NSAV',
  'certainty': 68},
 'CSGINTERNATIONAL': {'name': 'SGI INTERNATIONAL',
  'ticker': 'SGII',
  'certainty': 91},
 'HEXCELCORP': {'name': 'Hexcel Corp', 'ticker': 'HXL', 'certainty': 95},
 'SERVICENOW': {'name': 'ServiceNow Inc', 'ticker': 'NOW', 'certainty': 95},
 'TRIPOINTEHOMES': {'name': 'TRI Pointe Group Inc',
  'ticker': 'TPH',
  'certainty': 65},
 'AEGON': {'name': 'Aegon NV', 'ticker': 'AEG', 'certainty': 90},
 'NORTHRIMBANK': {'name': 'Northrim BanCorp Inc

In [614]:
def get_company_date_ranges(dataframe, company_list, company_ticker_dict):
    """Finds earliest and latest date for each company"""
    
    for company in company_list:
        df = dataframe[dataframe.username == company]
        min_date = np.min(df.time)
        max_date = np.max(df.time)
        company_ticker_dict[company]['Earliest Date'] = min_date
        company_ticker_dict[company]['Latest Date'] = max_date
        
    return company_ticker_dict

In [615]:
company_ticker_dict = get_company_date_ranges(dff, physical_companies, company_ticker_dict.copy())

In [616]:
company_ticker_dict['VMWARE']

{'name': 'VMware Inc',
 'ticker': 'VMW',
 'certainty': 90,
 'Earliest Date': datetime.date(2015, 1, 3),
 'Latest Date': datetime.date(2018, 7, 16)}

In [617]:
import pickle
with open('company_ticker_dict.pickle', 'wb') as handle:
    pickle.dump(company_ticker_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('company_ticker_dict.pickle', 'rb') as handle:
#     company_ticker_dict = pickle.load(handle)

In [286]:
from datetime import datetime
import requests
import time 

def get_yahoo_finance_page(ticker):
    """Gets yahoo finance page containing EPS data"""
    #https://finance.yahoo.com/calendar/earnings?day=2019-10-09&symbol=GE
    
    now = datetime.now()
    year = now.year
    month = now.month
    day = now.day
    
    base = 'https://finance.yahoo.com/calendar/earnings?day='
    url = '{}{}-{:02d}-{:02d}&symbol={}'.format(base,year, month, day, ticker)
    print(url)
    
    status_code = 0
    fault_counter = 0
    
    while status_code != 200:        
        page = requests.get(url)
        status_code = page.status_code
        if status_code != 200:
            fault_counter += 1
            time.sleep(5)
        if fault_counter > 5:
            return None
        
    return page

In [618]:
from datetime import timedelta  

def get_table_on_page(page, min_date, max_date):
    """Parses EPS data from table"""
    
    import re
    from bs4 import BeautifulSoup
    
    earnings_table = {}
    earnings_table_sorted = {}
    date_list = []
    estimate = []
    reported = []
    surprise = []
    
    
    soup = BeautifulSoup(page.text, 'lxml')
    table = soup.findAll('tbody')
    
    if table:
        table = table[0]
        quarters = table.findAll('tr')
        counter = 1
        for quarter in quarters:
            ED = quarter.findAll('td', attrs={'aria-label':'Earnings Date'})[0].text
            temp = re.match('([A-Za-z]+\s\d+,\s[0-9]{4})',ED)[1]
            ED = datetime.strptime(temp, '%b %d, %Y').date()
            #print(min_date, ED, max_date)

            if ED > min_date and ED < max_date + delta(:  #ED < max_date
                earnings_table[counter] = {} 
                earnings_table[counter]['Report Date'] = ED
                EPS = quarter.findAll('td', attrs={'aria-label':'EPS Estimate'})[0].text
                if EPS == 'N/A' or EPS == '-':
                    EPS = 0.
                earnings_table[counter]['EPS Estimate'] = float(EPS)

                EPS = quarter.findAll('td', attrs={'aria-label':'Reported EPS'})[0].text
                if EPS == 'N/A' or EPS == '-':
                    EPS = 0.
                earnings_table[counter]['EPS Reported'] = float(EPS)

                EPS = quarter.findAll('td', attrs={'aria-label':'Surprise(%)'})[0].text
                if EPS == 'N/A' or EPS == '-':
                    EPS = 0.
                earnings_table[counter]['% Deviation'] = float(EPS)
                counter += 1
                
        for i in range(len(earnings_table), 0, -1):

            date_list.append(earnings_table[i]['Report Date'])
            estimate.append(earnings_table[i]['EPS Estimate'])
            reported.append(earnings_table[i]['EPS Reported'])
            surprise.append(earnings_table[i]['% Deviation'])
        
        return {'date': date_list, 'estimate':estimate, 'reported':reported, 'surprise':surprise}
    return None

In [620]:
page = get_yahoo_finance_page('TSLA')
get_table_on_page(page, datetime(2017,1,1).date(), datetime(2019,1,1).date())

https://finance.yahoo.com/calendar/earnings?day=2019-11-14&symbol=TSLA


{'date': [datetime.date(2017, 2, 22),
  datetime.date(2017, 5, 3),
  datetime.date(2017, 8, 2),
  datetime.date(2017, 11, 1),
  datetime.date(2018, 2, 7),
  datetime.date(2018, 5, 2),
  datetime.date(2018, 8, 1),
  datetime.date(2018, 10, 24),
  datetime.date(2019, 1, 30),
  datetime.date(2019, 4, 24),
  datetime.date(2019, 7, 24),
  datetime.date(2019, 10, 23),
  datetime.date(2020, 1, 28),
  datetime.date(2020, 4, 22),
  datetime.date(2020, 7, 22),
  datetime.date(2020, 10, 21)],
 'estimate': [-0.43,
  -0.81,
  -1.82,
  -2.29,
  -3.12,
  -3.58,
  -2.92,
  -0.19,
  2.2,
  -0.69,
  -0.36,
  -0.42,
  1.29,
  0.0,
  0.0,
  0.0],
 'reported': [-0.69,
  -1.33,
  -1.33,
  -2.92,
  -3.04,
  -3.35,
  -3.06,
  2.9,
  1.93,
  -2.9,
  -1.12,
  1.86,
  0.0,
  0.0,
  0.0,
  0.0],
 'surprise': [-60.47,
  -64.2,
  26.92,
  -27.51,
  2.56,
  6.42,
  -4.79,
  1626.32,
  -12.27,
  -320.29,
  -211.11,
  542.86,
  0.0,
  0.0,
  0.0,
  0.0]}

In [291]:
import time
def pull_EPS_data(company_dict):

    dlist = {}
    for company, v in company_dict.items():
        ticker = v['ticker']
        min_date = v['Earliest Date']
        max_date = v['Latest Date']
        print(company, ticker)
        time.sleep(0.51)
        page = get_yahoo_finance_page(ticker)
        table = get_table_on_page(page, min_date, max_date)
        #print(table)
        if table:
            dlist[company] = table
    return dlist

In [292]:
EPS_data = pull_EPS_data(company_ticker_dict)

ANSYSINC ANSS
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=ANSS
QUALCOMM QCOM
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=QCOM
UMBBANK None
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=None
GSK GSKY
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=GSKY
CTWTR TWTR
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=TWTR
IMAGESENSINGSYS ISNS
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=ISNS
CHICOPEESAVINGS PSAV
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=PSAV
CSGINTERNATIONAL SGII
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=SGII
HEXCELCORP GAXCQ
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=GAXCQ
SERVICENOW NOW
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=NOW
TRIPOINTEHOMES CNT PRA
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=CNT PRA
AEGON AEG
https://finance.yahoo.com/

SMARTFINAL SMBK
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=SMBK
ENTEGRISINC ENTG
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=ENTG
MBFINANCIALBANK MBIF
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=MBIF
SAFEGUARDSCIENTIFICS SFE
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=SFE
GAFISA GFASY
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=GFASY
PROSINC GPRO
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=GPRO
OLDDOMINIONFREIGHTLINE ODFL
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=ODFL
PANAMERICANSILVER MXOM
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=MXOM
ADVANCEAUTOPARTS AAP
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=AAP
NIKE NKE
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=NKE
DEXCOM DXCM
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=DXCM
LUMBERLIQUIDATORS MRGX


https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=KOF
PENSKECARS WRLS
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=WRLS
MERGEHEALTHCARE PNHT
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=PNHT
CARVERFEDERALSAVINGSBANK FSS
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=FSS
FORD ASFD
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=ASFD
OLDPOINT GEPT
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=GEPT
PETINSURANCE QBEIF
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=QBEIF
COMSCOREINC SCOR
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=SCOR
TRANSACTTECH FMNG
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=FMNG
CDWCORPORATION KPELF
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=KPELF
AGENUSBIO AGEN
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=AGEN
JUNIPERNETWORKS JNPR
https://finance.ya

https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=CYBR
CORELOGIC CLGX
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=CLGX
INSIGHTENTERPRISESINC BFEN
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=BFEN
FIRSTSAVINGSBANKNORTHWEST FFNW
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=FFNW
EVERTECINC EVTC
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=EVTC
USCELLULAR TIGO
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=TIGO
BROADWIND BWEN
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=BWEN
SHOPHQ1 BPCP
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=BPCP
VIVINTSOLAR VIOSF
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=VIOSF
EDAPTMSCOMPANY FNTT
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=FNTT
THESTREET TST
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=TST
GPSTRATEGIES WSSSQ
https://finance

MAZORROBOTICS MZOR
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=MZOR
CINCINNATIINSURANCE HIIQ
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=HIIQ
MYBANKWELL GBAN085PREFEA
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=GBAN085PREFEA
THENEWHOMECOMPANY None
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=None
INSPERITY NSP
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=NSP
PCM PCM
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=PCM
CITITRENDS CTRN
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=CTRN
PAYCHEX PAYX
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=PAYX
CAMDENLIVING SRZ
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=SRZ
BOOZALLEN AVLY
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=AVLY
PUTNAMBANKCT CBKN
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=CBKN
LIMONEIRA LMNR
https

...

https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=MTLS
MAGNEGAS MECAQ
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=MECAQ
NEONODETECHNOLOGIES XIDEQ
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=XIDEQ
WAYNEBANK DNSKF
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=DNSKF
MYNORTHEASTBANK NWBM
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=NWBM
NTCAREERS MCGI
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=MCGI
POLARISGOV CHRG
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=CHRG
DISNEYLANDPARISFR BNPQF
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=BNPQF
MIDATLANTICSPRINT ATTC
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=ATTC
PRODUOBE HERC
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=HERC
ARCDOCUMENTSOLUTIONS ARC
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=ARC
USATRUCKRECRUITING DK

PORTERNOVELLIPR NOVL
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=NOVL
GODIGITALMARKETING AVLD
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=AVLD
CARQUESTAUTOPARTS OSI
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=OSI
ONEKINGSLANE OPNN
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=OPNN
HACHCOMPANY FNTT
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=FNTT
A10NETWORKS FCTN
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=FCTN
ACDELCO None
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=None
MANTECHINTERNATIONALCORPORATION ACO
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=ACO
TESSERATECHNOLOGIES LBTE
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=LBTE
GARTNER IT
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=IT
SAILPOINT SAIL
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=SAIL
SEND

MYORMCO MYO
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=MYO
CCCGLOBAL DLLR
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=DLLR
IMPLANTDIRECTINTL ATTC
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=ATTC
GENEXPERTSYSTEM NEWN
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=NEWN
PHENOMENEX LRTC
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=LRTC
FOBALASER GPRM
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=GPRM
SCIEXNEWS DTHR
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=DTHR
CARLISLECW CARI
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=CARI
ADVANTAGEMRKTG None
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=None
SINFULCOLORS CCSI
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=CCSI
ULTIMAIIINDONESIA MBND
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=MBND
GROUP1AUTOBUSINESSSUPPORTC

LATRICOLOR1059 PSYS
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=PSYS
DESTINATIONXL DXLG
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=DXLG
LASUAVECITA1043 AVTQZ
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=AVTQZ
AVXCORP AVX
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=AVX
IHSMARKIT INFO
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=INFO
GOLDFIELDSLTD MXOM
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=MXOM
ULTIMAIIID ULFS
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=ULFS
COGGINFORDJAX PENX
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=PENX
AMISPHILADELPHIA DPHIQ
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=DPHIQ
POWEREDBYINAP SLRX
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=SLRX
GAMESTOPITALIA None
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=None
WESTLAKECHEMIC

COVANTA None
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=None
INTERGLOBETECHNOLOGIES KEOSD
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=KEOSD
DISCOVERYCOVE DSCY
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=DSCY
CBSSPORTINGCLUB LC
https://finance.yahoo.com/calendar/earnings?day=2019-11-13&symbol=LC


In [369]:
EPS_data['WALMART']

{'date': [datetime.date(2017, 2, 21),
  datetime.date(2017, 5, 18),
  datetime.date(2017, 8, 17),
  datetime.date(2017, 11, 16),
  datetime.date(2018, 2, 20),
  datetime.date(2018, 5, 17)],
 'estimate': [1.29, 0.96, 1.07, 0.97, 1.37, 1.12],
 'reported': [1.3, 1.0, 1.08, 1.0, 1.33, 1.14],
 'surprise': [0.78, 4.17, 0.93, 3.09, -2.92, 1.79]}

In [294]:
import pickle
with open('EPS_data.pickle', 'wb') as handle:
    pickle.dump(EPS_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('EPS_data.pickle', 'rb') as handle:
#     EPS_data = pickle.load(handle)

In [263]:
def get_stock_price_from_ticker(ticker, start_date, end_date):
    from datetime import datetime
    from datetime import date
    import time
    import requests
    import json
    
    base = 'https://api-v2.intrinio.com/securities/{}/prices'.format(ticker)
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')
    #print(start_date, end_date)
    
    params1={'api_key':'Ojg5ZDFmOGNmMzJiOWZjM2RjZGNhNDRiM2JiNWJkM2M0',
                                    'start_date': start_date,
                                    'end_date': end_date,
                                     'frequency': 'daily',
                                    'page_size': 10000}
    
    def run_request(params):
        time.sleep(0.51)
        page = requests.get(base, params)
        return page
    
    status_code = 0
    limit = 0
    
    while status_code != 200:
        page = run_request(params1)
        status_code = page.status_code
        if status_code != 200:
            limit += 1
            time.sleep(5)
        if limit > 5:
            print('tried request 5+ times, breaking')
            return None
    #print(page.status_code)

    result = json.loads(page.text)
    if result:
        result = result['stock_prices']
        result.reverse()
        return result
    else:
        return None

In [266]:
import time
def fill_in_stock_price(dataframe, company_dict):
    """Uses intrinio API to get EOD stock price for each company, adding that data to the dataframe"""

    company_date_price_dict = {}
    
    for company, values in company_dict.items():
        print(company)
        
        ticker = values['ticker']
        min_date = values['Earliest Date']
        max_date = values['Latest Date']
        
        if ticker == None:
            continue
            
        df = dataframe[dataframe.username == company]
        date_list = list(df.time)
        date_list.sort(reverse=True)
        #print(np.min(date_list), np.max(date_list))
        
        result = get_stock_price_from_ticker(ticker, min_date, max_date)
        
        if not result:
            continue
        
        api_date_list = []
        stock_price_list = []
        
        for day_stats in result:
            date1 = datetime.strptime(day_stats['date'], '%Y-%m-%d').date()          
            stock_price_list.append(day_stats['close'])
            api_date_list.append(date1)
        
        api_date_list2 = []
        stock_price_list2 = []
        for date in date_list:
            if date in api_date_list:
                index = api_date_list.index(date)
                stock_price_list2.append(stock_price_list[index])
                api_date_list2.append(date)
            else:
                stock_price_list2.append(np.nan)
                api_date_list2.append(date)
            
        company_date_price_dict[company] = {}
        company_date_price_dict[company]['time'] = api_date_list2
        company_date_price_dict[company]['price'] = stock_price_list2
    
    return company_date_price_dict

In [267]:
company_date_price_dict = fill_in_stock_price(dff.copy(), company_ticker_dict)

ANSYSINC
QUALCOMM
UMBBANK
GSK
CTWTR
IMAGESENSINGSYS
CHICOPEESAVINGS
CSGINTERNATIONAL
HEXCELCORP
SERVICENOW
TRIPOINTEHOMES
AEGON
NORTHRIMBANK
tried request 5+ times, breaking
ADDUSHOMECARE
PRIMOWATERCORP
BRANDYWINEREALTYTRUST
LIGHTINTHEBOX
MYHORIZONBANK
OCCSOLUTIONS
CAPITALSENIORLIVING
EXELIS
BUSEYBANK
AIRPRODUCTSCAREERS
tried request 5+ times, breaking
PHIINC
STAMPS.COM
MARCHEX
RFMDMEDIA
tried request 5+ times, breaking
ICFINTERNATIONAL
APPLIEDGENETICTECHNOLOGIESCORPORATION
LENNOX
ABBVIECAREERS
SABRECORPORATION
FARMERSBANKGROUP
POWERINTEGRATIONS
REXNORDCORPORATE
HFRECIPES
EFI.DIGITAL.PRINT.TECHNOLOGY
CHILDRENSPLACE
RESOURCESGLOBALPROFESSIONALS
OMAPLAZA
ALIMERASCIENCES
ASHLANDINC
GREENBANKNA
ROCKLANDTRUST
tried request 5+ times, breaking
IDENTIV
PENNYMACUSA
tried request 5+ times, breaking
VALEROENERGY
WALMART
KAPSTONEPAPER
ENGILITY
EXPLORE.IHS
HARTEHANKS
PNCBANK
WILLISGROUP
STARBUCKSINDIA
UNITEDCOMMUNITYBANKIN
HIBBETTSPORTS
EQUINIX
WAGEWORKS
SEQUANS
HEARTLANDHPY
USGCORP
TERADATA
NEWSCO

AMERICANAIRLINES
DORMANPRODUCTS
ES.TOPRURAL
ESPN
HOMEAWAY.FEWO.DIREKT
VIATORTOURS
TRIPADVISOR
Z2GAMES
FLIPKEY
PROCTERGAMBLE
ONDECKCAPITAL
BLUEAPRON
YAHOO
ROYALCARIBBEAN
VIAVISOLUTIONS
FOOTLOCKER
MINDBODYONLINE
PLUGPOWERINC
GODADDY
LENDINGCLUBTEAM
FREDMEYER
KROGER
BLOOMINGDALES
PLANETFITNESS
LEFFE.UK
ASKDOTCOM
ILIKETWOO
ABOUTDOTCOM
MINDSPARKINTERACTIVE
THEPRINCETONREVIEW
MHELPDESK
tried request 5+ times, breaking
DEVRYUNIVERSITY
PDR.NETWORK
MEDPAGETODAY
LOWES
LODESTONEMANAGEMENTCONSULTANTS
XOOM
DWELLSTUDIO
SIMPLYMAC
OFFICIALGAMEINFORMER
ORACLE
LOFT
MAURICES
SALLIEMAE
QAUTOHASIT
KOHLS
SYSCOCORPORATION
MENSWEARHOUSE
IDEXX
CISCO
SMITHS
KWIKSHOPSTORES
FRYSFOODSTORES
LOAFNJUGSTORES
MYPUBLISHER
THISLIFEAPP
BORROWLENSES
HAWAIIANAIRLINES
POOLCORP1
PICKNSAVESTORES
SPREESPARKLING
SHASTASODA
MARIANOSMARKET
CRESTLINERBOATS
AMERICANWOODMARK
NORTHPOINTMAZDA
RAYGLASSNZ
COGGINDELANDLINCOLN
COURTESYPALMHARBORHONDA
NALLEYLEXUSGALLERIA
MOORMANSSHOWTEC
COGGINHONDAORLANDO
ATAGLANCE
NORTHPOINTTOYOTASCION
COG

BLUECHIPCASINO
GUNLOCKE
IPCASINO
AMELIABELLECASINO
SAMSTOWNTUNICA
DELTADOWNSCASINO
EVANGELINEDOWNSRACETRACKCASINO
KANSASSTAR
SUNCOASTCASINO
DIAMONDJOWORTH
TREASURECHESTCASINO
BORGATAHOTELCASINOANDSPA
PARADICE
ORLEANSCASINO
ELDORADOCASINO
BCONNECTEDSOCIAL
CALIFORNIACASINO
SAMSTOWNLV
FREMONTCASINO
tried request 5+ times, breaking
SAMSTOWN
EASTPAK
NAPAPIJRI
MAJESTICATHLETIC
NEENAHFINEPAPER
NAUTICA
NEOGENCORP
VIRCOINC
SMARTWOOL
FRANKLINCOVEY
SPAMBRAND
HORMEL.CHILI
CURE81HAM
SKIPPY
CLUBDONAMARIA
CREE
ENERGOUS
INTERFACE
SPCAPITALIQ
MEDCOTOOL
UNITEDSTATIONERS
WOWT6NEWS
ZIGGOCOMPANY
THEMEDIAKITCHEN
MARINEMAXTABLEROCKLAKE
LAMARADVERTISING
tried request 5+ times, breaking
MARINEMAXMINNESOTA
WITNTV
MARINEMAXHUNTINGTON
72ANDSUNNY
1011NEWS
MARINEMAXPOMPANOBEACH
MARINEMAXSHIPBOTTOM
MARINEMAXNAPLES
AUTOCLUBSPEEDWAY
23WIFR
MARINEMAXFISHING
WKYTTV
MARINEMAXPENSACOLAFL
NEWSCENTER16
MARINEMAXCONNECTICUT
NEWS7
COLLEMCVOY
UNITEDHEALTHCARE
KCWY13
VITROAGENCY
MARINEMAXOFGEORGIA
KKTV11NEWS
SANFORDORLANDOKC
KB

MELLANOXTECH
INFINERA
CYMERINC
ASML
ZENDESK
BLACKLINEHQ
JET
INTELFPGA
PIVOTALSOFTWARE
CEMENTOSPACASMAYOPERU
ABFTODAY
ATLASSIAN
LENNARMULTIFAMILYLIVING
FOX40
NORTHAMERICANTITLE
EAGLEHOMEMORTGAGE
THESFSHIPYARD
VALENCIACALIFORNIA
GREATPARKNEIGHBORHOODS
DENTONRC
DMNGUIDELIVE
AETHLONMEDICAL
AELPJUNEAU
DMNMEDIASOLUTIONS
AKERSBIO
BIOTELEMETRYINC
AVXCORPORATION
tried request 5+ times, breaking
BLUEBIRDCORPORATION
AVISTAUTILITIES
tried request 5+ times, breaking
AGILENT.TECH
ABINSIGHTS
tried request 5+ times, breaking
ALCONEYECARE
tried request 5+ times, breaking
ALDIADALLAS
ING
DALLASMORNINGNEWS
BANCOFCALIFORNIA
CRYOLIFEINC
FARMBUREAUFINANCIALSERVICES
tried request 5+ times, breaking
AMERICANINTERCONTINENTALUNIVERSITY
CYTOKINETICSINC
CENTRALVALLEYCOMMUNITYBANK
EAGLEBANKCORP
COLORADOTECH
CORECIVIC
CUSTOMERSBANK
WESBANCO
ECHOLOGISTICS
DONEGALGROUP
CAMDENNATIONAL
CAPFED
CCBIDAHO
FIRSTINTERNETBANK
MYBANK
DRIVEFAF
FORUMENERGYTECH
FIRSTFINANCIALBANKNA
MWBANK
GEOGROUP
DRIVECST
HAEMONETICSCORP
HBFULLE

tried request 5+ times, breaking
TIGERDIRECT
MACATAWABANK
ANTHEMWORLDWIDE
SCHAWKINC
EMCINSURANCE
MTBCORP
tried request 5+ times, breaking
MONROEBANKTRUST
MITEKSYSTEMS
TOWNEBANK
MIRATITX
THELEADERCORNINGNY
CONSOLIDATEDCCI
MIDDLESEXWATER
NATURESSUNSHINE
NATURALALTERNATIVESINTERNATIONALINC
MATRIXMEDICALNETWORK
PROFIREENERGY
COMMUNITYBT
FLORIDAPNEUMATIC
QCBANK
COLDEEZE
ROCKFORDBANKANDTRUST
LOGISTICARE
LUZERNEBANK
NEWYORKSPORTSCLUBS
SANDYSPRINGBANK
SEASPINE
ISHARESETFS
ITURANUSA
BOSTONSPORTSCLUBS
MATERIALISENV
MAGNEGAS
NEONODETECHNOLOGIES
WAYNEBANK
MYNORTHEASTBANK
NTCAREERS
POLARISGOV
DISNEYLANDPARISFR
MIDATLANTICSPRINT
PRODUOBE
ARCDOCUMENTSOLUTIONS
USATRUCKRECRUITING
UNITEDBANKKY02
UNITEDBANKKY03
STARTRON
STARBRITESOLUTIONS
OMNICELLINC
PSBNEPA
GAMEHOUSE
BAMKO.US
PEOPLESBANK1864
CITIZENSFIRSTBANK
WILLIAMLYONHOMES
THESOURCECANADA
SALLYBEAUTYCHILE
BELLMEDIASALES
WILLAMETTEVALLEYVINEYARDS
OGXBEAUTY
OSTERIAPHILADELPHIA
RESMEDUS
FISHERPRICE
L3NARDAMICROWAVEWEST
WILSONSLEATHER
COLEMANUSA
RESMEDAM

In [324]:
company_date_price_dict['FREDSINC']

{'time': [datetime.date(2018, 7, 17),
  datetime.date(2018, 7, 14),
  datetime.date(2018, 7, 11),
  datetime.date(2018, 7, 5),
  datetime.date(2018, 6, 14),
  datetime.date(2018, 6, 8),
  datetime.date(2018, 6, 7),
  datetime.date(2018, 6, 5),
  datetime.date(2018, 6, 2),
  datetime.date(2018, 6, 1),
  datetime.date(2018, 5, 29),
  datetime.date(2018, 5, 28),
  datetime.date(2018, 5, 27),
  datetime.date(2018, 5, 26),
  datetime.date(2018, 5, 23),
  datetime.date(2018, 5, 22),
  datetime.date(2018, 5, 21),
  datetime.date(2018, 5, 20),
  datetime.date(2018, 5, 19),
  datetime.date(2018, 5, 18),
  datetime.date(2018, 5, 17),
  datetime.date(2018, 5, 16),
  datetime.date(2018, 5, 15),
  datetime.date(2018, 5, 14),
  datetime.date(2018, 5, 13),
  datetime.date(2018, 5, 12),
  datetime.date(2018, 5, 11),
  datetime.date(2018, 5, 10),
  datetime.date(2018, 5, 9),
  datetime.date(2018, 5, 8),
  datetime.date(2018, 5, 7),
  datetime.date(2018, 5, 6),
  datetime.date(2018, 5, 5),
  datetime.da

In [268]:
import pickle
with open('company_date_price_dict.pickle', 'wb') as handle:
    pickle.dump(company_date_price_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('company_date_price_dict.pickle', 'rb') as handle:
#     company_date_price_dict = pickle.load(handle)

In [398]:
import requests
import simplejson as json
import time
def find_true_financial_quarter(ticker, date1):
    
    year = date1.year
    month = date1.month
    day = date1.day
    
    #guess quarter
    if month in [1, 2, 3]:
        quarter = 1
    elif month in [4, 5, 6]:
        quarter = 2
    elif month in [7, 8, 9]:
        quarter = 3
    elif month in [10, 11, 12]:
        quarter = 4
                        
    
    def make_request(url):    
        status = 0
        retry = 0
        while status != 200:
            time.sleep(0.51)
            page = requests.get(url, params={'api_key':'Ojg5ZDFmOGNmMzJiOWZjM2RjZGNhNDRiM2JiNWJkM2M0'})
            status = page.status_code
            retry += 1
            if retry == 4:
                time.sleep(4)
            if retry > 5:
                print('Retry failed 5 times with code ', status)
                return None
        return page
    
    #check proper quarter
    bounce_back = 0
    while True:
        financial_string = '{}-{}-{}-Q{}'.format(ticker, 'income_statement', year, quarter)
        url = 'https://api-v2.intrinio.com/fundamentals/{}/standardized_financials'.format(financial_string)
        page = make_request(url)  
        if page is None:
            return None
        result = json.loads(page.text)

        check_quarter =  int(result['fundamental']['fiscal_period'][1])
        start_date = result['fundamental']['start_date']
        end_date = result['fundamental']['end_date']
        start_date = datetime.strptime(start_date,'%Y-%m-%d').date()
        end_date = datetime.strptime(end_date,'%Y-%m-%d').date()
        
        if date1 > end_date:
            quarter += 1
            bounce_back += 1
        elif date1 < start_date:
            quarter -= 1
            bounce_back += 1
        else:
            break
        if bounce_back > 2:
            return None
        if quarter > 4:
            quarter = 1
            year += 1
        if quarter < 1:
            quarter = 4
            year -= 1
        #print(quarter, date1)
        #print(start_date, end_date)
            
    return [quarter, start_date, end_date]

In [382]:
find_true_financial_quarter('VMWARE', datetime(2017, 10, 27).date())

Retry failed 5 times with code  404


In [399]:
from datetime import timedelta  
def get_company_quarters(company_dict):
    
    
    company_quarters = {}
    for company, value in company_dict.items():
        
        print(company)
        min_date = value['Earliest Date']
        max_date = value['Latest Date']
        ticker = value['ticker']
        #print(ticker, min_date, max_date)
        
        if ticker != None:
            
            company_quarters[company] = []
            while min_date < max_date:
                result = find_true_financial_quarter(ticker, min_date)
                print(result)
                if result:
                    [quarter, start_date, end_date] = result
                    company_quarters[company].append((start_date, end_date))
                    min_date = end_date + timedelta(days=2)
                else:
                    break
                
        
    return company_quarters
        

In [400]:
# get_company_quarters({'VMWARE':{'name': 'VMware Inc',
#  'ticker': 'VMW',
#  'certainty': 100,
#  'Earliest Date': datetime(2015, 1, 3).date(),
#  'Latest Date': datetime(2018, 7, 16).date()}})

In [401]:
company_quarter_range = get_company_quarters(company_ticker_dict)

ANSYSINC
[1, datetime.date(2015, 1, 1), datetime.date(2015, 3, 31)]
[2, datetime.date(2015, 4, 1), datetime.date(2015, 6, 30)]
[3, datetime.date(2015, 7, 1), datetime.date(2015, 9, 30)]
[4, datetime.date(2015, 10, 1), datetime.date(2015, 12, 31)]
[1, datetime.date(2016, 1, 1), datetime.date(2016, 3, 31)]
[2, datetime.date(2016, 4, 1), datetime.date(2016, 6, 30)]
[3, datetime.date(2016, 7, 1), datetime.date(2016, 9, 30)]
[4, datetime.date(2016, 10, 1), datetime.date(2016, 12, 31)]
[1, datetime.date(2017, 1, 1), datetime.date(2017, 3, 31)]
[2, datetime.date(2017, 4, 1), datetime.date(2017, 6, 30)]
[3, datetime.date(2017, 7, 1), datetime.date(2017, 9, 30)]
[4, datetime.date(2017, 10, 1), datetime.date(2017, 12, 31)]
[1, datetime.date(2018, 1, 1), datetime.date(2018, 3, 31)]
[2, datetime.date(2018, 4, 1), datetime.date(2018, 6, 30)]
[3, datetime.date(2018, 7, 1), datetime.date(2018, 9, 30)]
QUALCOMM
[2, datetime.date(2014, 12, 29), datetime.date(2015, 3, 29)]
[3, datetime.date(2015, 3, 30)

[4, datetime.date(2015, 10, 1), datetime.date(2015, 12, 31)]
[1, datetime.date(2016, 1, 1), datetime.date(2016, 3, 31)]
[2, datetime.date(2016, 4, 1), datetime.date(2016, 6, 30)]
[3, datetime.date(2016, 7, 1), datetime.date(2016, 9, 30)]
[4, datetime.date(2016, 10, 1), datetime.date(2016, 12, 31)]
[1, datetime.date(2017, 1, 1), datetime.date(2017, 3, 31)]
[2, datetime.date(2017, 4, 1), datetime.date(2017, 6, 30)]
[3, datetime.date(2017, 7, 1), datetime.date(2017, 9, 30)]
[4, datetime.date(2017, 10, 1), datetime.date(2017, 12, 31)]
[1, datetime.date(2018, 1, 1), datetime.date(2018, 3, 31)]
[2, datetime.date(2018, 4, 1), datetime.date(2018, 6, 30)]
[3, datetime.date(2018, 7, 1), datetime.date(2018, 9, 30)]
STAMPS.COM
[1, datetime.date(2015, 1, 1), datetime.date(2015, 3, 31)]
[2, datetime.date(2015, 4, 1), datetime.date(2015, 6, 30)]
[3, datetime.date(2015, 7, 1), datetime.date(2015, 9, 30)]
[4, datetime.date(2015, 10, 1), datetime.date(2015, 12, 31)]
[1, datetime.date(2016, 1, 1), dateti

[2, datetime.date(2018, 4, 1), datetime.date(2018, 6, 30)]
[3, datetime.date(2018, 7, 1), datetime.date(2018, 9, 30)]
PENNYMACUSA
[2, datetime.date(2014, 12, 1), datetime.date(2015, 2, 28)]
Retry failed 5 times with code  404
None
VALEROENERGY
Retry failed 5 times with code  404
None
WALMART
None
KAPSTONEPAPER
Retry failed 5 times with code  404
None
ENGILITY
[1, datetime.date(2015, 1, 1), datetime.date(2015, 3, 31)]
[2, datetime.date(2015, 4, 1), datetime.date(2015, 6, 30)]
[3, datetime.date(2015, 7, 1), datetime.date(2015, 9, 30)]
[4, datetime.date(2015, 10, 1), datetime.date(2015, 12, 31)]
[1, datetime.date(2016, 1, 1), datetime.date(2016, 4, 1)]
[2, datetime.date(2016, 4, 2), datetime.date(2016, 7, 1)]
[3, datetime.date(2016, 7, 1), datetime.date(2016, 9, 30)]
[4, datetime.date(2016, 10, 1), datetime.date(2016, 12, 31)]
[1, datetime.date(2017, 1, 1), datetime.date(2017, 3, 31)]
[2, datetime.date(2017, 4, 1), datetime.date(2017, 6, 30)]
[3, datetime.date(2017, 7, 1), datetime.date(2

[3, datetime.date(2016, 6, 12), datetime.date(2016, 9, 3)]
[4, datetime.date(2016, 9, 4), datetime.date(2016, 12, 31)]
[1, datetime.date(2017, 1, 1), datetime.date(2017, 3, 25)]
[2, datetime.date(2017, 3, 26), datetime.date(2017, 6, 17)]
[3, datetime.date(2017, 6, 18), datetime.date(2017, 9, 9)]
[4, datetime.date(2017, 9, 10), datetime.date(2017, 12, 30)]
[1, datetime.date(2017, 12, 31), datetime.date(2018, 3, 24)]
[2, datetime.date(2018, 3, 25), datetime.date(2018, 6, 16)]
[3, datetime.date(2018, 6, 17), datetime.date(2018, 9, 8)]
GUESS
None
FIRST.AMERICAN.FINANCIAL
Retry failed 5 times with code  404
None
AMEDICACORPORATION
Retry failed 5 times with code  404
None
CROCS
[1, datetime.date(2015, 1, 1), datetime.date(2015, 3, 31)]
[2, datetime.date(2015, 4, 1), datetime.date(2015, 6, 30)]
[3, datetime.date(2015, 7, 1), datetime.date(2015, 9, 30)]
[4, datetime.date(2015, 10, 1), datetime.date(2015, 12, 31)]
[1, datetime.date(2016, 1, 1), datetime.date(2016, 3, 31)]
[2, datetime.date(2016

[1, datetime.date(2015, 1, 4), datetime.date(2015, 4, 25)]
[2, datetime.date(2015, 4, 26), datetime.date(2015, 7, 18)]
[3, datetime.date(2015, 7, 19), datetime.date(2015, 10, 10)]
[4, datetime.date(2015, 10, 11), datetime.date(2016, 1, 2)]
[1, datetime.date(2016, 1, 3), datetime.date(2016, 4, 23)]
[2, datetime.date(2016, 4, 24), datetime.date(2016, 7, 16)]
[3, datetime.date(2016, 7, 17), datetime.date(2016, 10, 8)]
[4, datetime.date(2016, 10, 9), datetime.date(2016, 12, 31)]
[1, datetime.date(2017, 1, 1), datetime.date(2017, 4, 22)]
[2, datetime.date(2017, 4, 23), datetime.date(2017, 7, 15)]
[3, datetime.date(2017, 7, 16), datetime.date(2017, 10, 7)]
[4, datetime.date(2017, 10, 8), datetime.date(2017, 12, 30)]
[1, datetime.date(2017, 12, 31), datetime.date(2018, 4, 21)]
[2, datetime.date(2018, 4, 22), datetime.date(2018, 7, 14)]
[3, datetime.date(2018, 7, 15), datetime.date(2018, 10, 6)]
NIKE
[3, datetime.date(2014, 12, 1), datetime.date(2015, 2, 28)]
None
DEXCOM
[1, datetime.date(2015

...

Retry failed 5 times with code  404
None
MAGNEGAS
Retry failed 5 times with code  404
None
NEONODETECHNOLOGIES
Retry failed 5 times with code  404
None
WAYNEBANK
Retry failed 5 times with code  404
None
MYNORTHEASTBANK
Retry failed 5 times with code  404
None
NTCAREERS
Retry failed 5 times with code  404
None
POLARISGOV
Retry failed 5 times with code  404
None
DISNEYLANDPARISFR
Retry failed 5 times with code  404
None
MIDATLANTICSPRINT
[1, datetime.date(2017, 1, 1), datetime.date(2017, 3, 31)]
[2, datetime.date(2017, 4, 1), datetime.date(2017, 6, 30)]
[3, datetime.date(2017, 7, 1), datetime.date(2017, 9, 30)]
[4, datetime.date(2017, 10, 1), datetime.date(2017, 12, 31)]
[1, datetime.date(2018, 1, 1), datetime.date(2018, 3, 31)]
[2, datetime.date(2018, 4, 1), datetime.date(2018, 6, 30)]
[3, datetime.date(2018, 7, 1), datetime.date(2018, 9, 30)]
PRODUOBE
Retry failed 5 times with code  404
None
ARCDOCUMENTSOLUTIONS
[1, datetime.date(2017, 1, 1), datetime.date(2017, 3, 31)]
[2, datetime.da

[3, datetime.date(2017, 2, 1), datetime.date(2017, 4, 30)]
[4, datetime.date(2017, 5, 1), datetime.date(2017, 7, 31)]
[1, datetime.date(2017, 8, 1), datetime.date(2017, 10, 31)]
[2, datetime.date(2017, 11, 1), datetime.date(2018, 1, 31)]
[3, datetime.date(2018, 2, 1), datetime.date(2018, 4, 30)]
[4, datetime.date(2018, 5, 1), datetime.date(2018, 7, 31)]
DEUTSCHEBANK
Retry failed 5 times with code  404
None
TAYLORDEVICES
DUKE.ENERGY
[2, datetime.date(2017, 4, 1), datetime.date(2017, 6, 30)]
[3, datetime.date(2017, 7, 1), datetime.date(2017, 9, 30)]
[4, datetime.date(2017, 10, 1), datetime.date(2017, 12, 31)]
[1, datetime.date(2018, 1, 1), datetime.date(2018, 3, 31)]
[2, datetime.date(2018, 4, 1), datetime.date(2018, 6, 30)]
[3, datetime.date(2018, 7, 1), datetime.date(2018, 9, 30)]
AMERICANADDICTIONCENTERS
[2, datetime.date(2017, 4, 1), datetime.date(2017, 6, 30)]
[3, datetime.date(2017, 7, 1), datetime.date(2017, 9, 30)]
[4, datetime.date(2017, 10, 1), datetime.date(2017, 12, 31)]
[1, 

[3, datetime.date(2018, 7, 1), datetime.date(2018, 9, 30)]
XBIOTECH
[2, datetime.date(2017, 4, 1), datetime.date(2017, 6, 30)]
[3, datetime.date(2017, 7, 1), datetime.date(2017, 9, 30)]
[4, datetime.date(2017, 10, 1), datetime.date(2017, 12, 31)]
[1, datetime.date(2018, 1, 1), datetime.date(2018, 3, 31)]
[2, datetime.date(2018, 4, 1), datetime.date(2018, 6, 30)]
[3, datetime.date(2018, 7, 1), datetime.date(2018, 9, 30)]
TRUNKCLUB
Retry failed 5 times with code  404
None
MOLECULARDEVICES
Retry failed 5 times with code  404
None
SPARTANMOTORSINC
[2, datetime.date(2017, 4, 1), datetime.date(2017, 6, 30)]
[3, datetime.date(2017, 7, 1), datetime.date(2017, 9, 30)]
[4, datetime.date(2017, 10, 1), datetime.date(2017, 12, 31)]
[1, datetime.date(2018, 1, 1), datetime.date(2018, 3, 31)]
[2, datetime.date(2018, 4, 1), datetime.date(2018, 6, 30)]
[3, datetime.date(2018, 7, 1), datetime.date(2018, 9, 30)]
PARAMOUNT
Retry failed 5 times with code  404
None
ALLIANCEHEALTHCARESERVICES
[2, datetime.dat

Retry failed 5 times with code  404
None
TECHNIPFMC
[3, datetime.date(2017, 7, 1), datetime.date(2017, 9, 30)]
[4, datetime.date(2017, 10, 1), datetime.date(2017, 12, 31)]
[1, datetime.date(2018, 1, 1), datetime.date(2018, 3, 31)]
[2, datetime.date(2018, 4, 1), datetime.date(2018, 6, 30)]
[3, datetime.date(2018, 7, 1), datetime.date(2018, 9, 30)]
CITYNATIONALBANK
Retry failed 5 times with code  404
None
STATEAUTO
Retry failed 5 times with code  404
None
LEGGMASON
None
AMERICANAXLE
BOISECASCADECOMPANY
[3, datetime.date(2017, 7, 1), datetime.date(2017, 9, 30)]
[4, datetime.date(2017, 10, 1), datetime.date(2017, 12, 31)]
[1, datetime.date(2018, 1, 1), datetime.date(2018, 3, 31)]
[2, datetime.date(2018, 4, 1), datetime.date(2018, 6, 30)]
[3, datetime.date(2018, 7, 1), datetime.date(2018, 9, 30)]
CALLAWAYPREOWNED
Retry failed 5 times with code  404
None
TOULONDESIGN
Retry failed 5 times with code  404
None
CBS6NEWS
[1, datetime.date(2017, 7, 1), datetime.date(2017, 9, 30)]
[2, datetime.date

Retry failed 5 times with code  404
None
DLVBBDO
Retry failed 5 times with code  404
None
MARKBBDO
99ENTERPRISES
Retry failed 5 times with code  404
None
TELIAPAVLABBDO
Retry failed 5 times with code  404
None
PROXIMITYLONDON
Retry failed 5 times with code  404
None
HELLOTRAFFIK
Retry failed 5 times with code  404
None
BBDOUKRAINE
Retry failed 5 times with code  404
None
BBDOWORLDWIDE
Retry failed 5 times with code  404
None
TBWAWORLDWIDE
Retry failed 5 times with code  404
None
REDHANDEDAGENCY
Retry failed 5 times with code  404
None
ALMAPBBDO
Retry failed 5 times with code  404
None
GITAM.BBDO
Retry failed 5 times with code  404
None
BBDOHONDURAS
Retry failed 5 times with code  404
None
APEXPUBLICIDAD
Retry failed 5 times with code  404
None
GARNIERBBDO
Retry failed 5 times with code  404
None
BBDOGERMANY
PROXIMITYMADRID
Retry failed 5 times with code  404
None
BBDOCONNECT
Retry failed 5 times with code  404
None
BBDOPORTUGAL
Retry failed 5 times with code  404
None
PROXIMITYBBDOPARI

In [407]:
company_quarter_range['WENDYS']

KeyError: 'WENDYS'

In [402]:
import pickle
with open('company_quarter_range.pickle', 'wb') as handle:
    pickle.dump(company_quarter_range, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('company_quarter_range.pickle', 'rb') as handle:
#     company_quarter_range = pickle.load(handle)

In [411]:
company_quarter_range

{'ANSYSINC': [(datetime.date(2015, 1, 1), datetime.date(2015, 3, 31)),
  (datetime.date(2015, 4, 1), datetime.date(2015, 6, 30)),
  (datetime.date(2015, 7, 1), datetime.date(2015, 9, 30)),
  (datetime.date(2015, 10, 1), datetime.date(2015, 12, 31)),
  (datetime.date(2016, 1, 1), datetime.date(2016, 3, 31)),
  (datetime.date(2016, 4, 1), datetime.date(2016, 6, 30)),
  (datetime.date(2016, 7, 1), datetime.date(2016, 9, 30)),
  (datetime.date(2016, 10, 1), datetime.date(2016, 12, 31)),
  (datetime.date(2017, 1, 1), datetime.date(2017, 3, 31)),
  (datetime.date(2017, 4, 1), datetime.date(2017, 6, 30)),
  (datetime.date(2017, 7, 1), datetime.date(2017, 9, 30)),
  (datetime.date(2017, 10, 1), datetime.date(2017, 12, 31)),
  (datetime.date(2018, 1, 1), datetime.date(2018, 3, 31)),
  (datetime.date(2018, 4, 1), datetime.date(2018, 6, 30)),
  (datetime.date(2018, 7, 1), datetime.date(2018, 9, 30))],
 'QUALCOMM': [(datetime.date(2014, 12, 29), datetime.date(2015, 3, 29)),
  (datetime.date(2015, 

In [442]:
def link_quarter_to_eps_release(EPS_data, quarter_data):
    """Find date closest to end of a quarter which corresponds with the release of EPS"""
    company_quarter_eps_link = {}
    for company, value_dict in EPS_data.items():
        print(company)
        if value_dict:
            eps_releases = value_dict['date']
            quarter_dates = quarter_data[company]
            end_quarter_dates = [i[1] for i in quarter_dates]

            #print(eps_releases)
            #print(quarter_dates)
            #print(end_quarter_dates)
            if eps_releases and end_quarter_dates:
                company_quarter_eps_link[company] = {}
                for index, current_eps_release_date in enumerate(eps_releases):
                    distance_from = [(current_eps_release_date - i).days for i in end_quarter_dates]
                    print(distance_from)
                    for i, num in enumerate(distance_from):
                        if num >= 0:
                            closest_index = i
                    print(closest_index)
                    #print(end_quarter_dates[closest_index])
                    company_quarter_eps_link[company][end_quarter_dates[closest_index]] = current_eps_release_date
                print(company_quarter_eps_link[company])   
            #break
        
    return company_quarter_eps_link
    

In [443]:
link_quarter_to_eps_release(EPS_data, company_quarter_range)

ANSYSINC
[695, 604, 512, 420, 329, 238, 146, 54, -36, -127, -219, -311, -401, -492, -584]
7
[764, 673, 581, 489, 398, 307, 215, 123, 33, -58, -150, -242, -332, -423, -515]
8
[855, 764, 672, 580, 489, 398, 306, 214, 124, 33, -59, -151, -241, -332, -424]
9
[946, 855, 763, 671, 580, 489, 397, 305, 215, 124, 32, -60, -150, -241, -333]
10
[1058, 967, 875, 783, 692, 601, 509, 417, 327, 236, 144, 52, -38, -129, -221]
11
[1128, 1037, 945, 853, 762, 671, 579, 487, 397, 306, 214, 122, 32, -59, -151]
12
{datetime.date(2016, 12, 31): datetime.date(2017, 2, 23), datetime.date(2017, 3, 31): datetime.date(2017, 5, 3), datetime.date(2017, 6, 30): datetime.date(2017, 8, 2), datetime.date(2017, 9, 30): datetime.date(2017, 11, 1), datetime.date(2017, 12, 31): datetime.date(2018, 2, 21), datetime.date(2018, 3, 31): datetime.date(2018, 5, 2)}
QUALCOMM
[752, 661, 570, 479, 388, 297, 206, 115, 24, -67, -158, -249, -340, -431, -529]
8
[843, 752, 661, 570, 479, 388, 297, 206, 115, 24, -67, -158, -249, -340, -4

[775, 684, 593, 502, 411, 320, 229, 131, 40, -51, -142]
8
[866, 775, 684, 593, 502, 411, 320, 222, 131, 40, -51]
9
{datetime.date(2016, 12, 24): datetime.date(2017, 2, 2), datetime.date(2017, 3, 25): datetime.date(2017, 5, 3), datetime.date(2017, 6, 24): datetime.date(2017, 8, 4), datetime.date(2017, 9, 30): datetime.date(2017, 12, 7), datetime.date(2017, 12, 30): datetime.date(2018, 2, 8), datetime.date(2018, 3, 31): datetime.date(2018, 5, 10)}
CAMPBELLEWALD
AUTOMATICDATAPROCESSING
[398, 307, 216, 124, 32, -58, -149, -241, -333, -423, -514, -606]
4
[489, 398, 307, 215, 123, 33, -58, -150, -242, -332, -423, -515]
5
[574, 483, 392, 300, 208, 118, 27, -65, -157, -247, -338, -430]
6
[672, 581, 490, 398, 306, 216, 125, 33, -59, -149, -240, -332]
7
[762, 671, 580, 488, 396, 306, 215, 123, 31, -59, -150, -242]
8
[853, 762, 671, 579, 487, 397, 306, 214, 122, 32, -59, -151]
9
{datetime.date(2016, 12, 31): datetime.date(2017, 2, 1), datetime.date(2017, 3, 31): datetime.date(2017, 5, 3), datetim

IndexError: list index out of range

In [ ]:
{'date': [datetime.date(2017, 2, 21),
  datetime.date(2017, 5, 18),
  datetime.date(2017, 8, 17),
  datetime.date(2017, 11, 16),
  datetime.date(2018, 2, 20),
  datetime.date(2018, 5, 17)],
 'estimate': [1.29, 0.96, 1.07, 0.97, 1.37, 1.12],
 'reported': [1.3, 1.0, 1.08, 1.0, 1.33, 1.14],
 'surprise': [0.78, 4.17, 0.93, 3.09, -2.92, 1.79]}

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

pipe = Pipeline(steps = ([('MagnitudeScaler', ),
                           ('RandomForest', RandomForestClassifier())]))

In [ ]:
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(pipe, param_grid={'n_estimators': [], })

gs.fit()